<a href="https://colab.research.google.com/github/Ximena5745/Developer/blob/main/Indicadores_Kawak.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
from google.colab import files
import ast

In [2]:
# Cargar archivo desde botón
uploaded = files.upload()

Saving consulta_indicadores (3).xlsx to consulta_indicadores (3) (1).xlsx


In [3]:
# Obtener el nombre del archivo cargado
filename = list(uploaded.keys())[0]

In [4]:
# Leer contenido del archivo Excel
Reporte_Indicadores = pd.read_excel(filename)

# Vista previa del DataFrame
print("Vista previa del archivo cargado:")
print(Reporte_Indicadores.head())


Vista previa del archivo cargado:
    ID                                         nombre       clasificacion  \
0   77       Disponibilidad de servicios tecnológicos  Estrat&eacute;gico   
1  121  Ejecución presupuestal de la nómina acumulada           Operativo   
2  124         Indice de Frecuencia de accidentalidad           Operativo   
3  125                            Indice de severidad           Operativo   
4  128                    Ausentísmo por causa médica           Operativo   

    sentido                                        proceso  \
0  Positivo  GESTIÓN DE LA TECNOLOGÍA E INNOVACIÓN DIGITAL   
1  Negativo        GESTIÓN Y DESARROLLO DEL TALENTO HUMANO   
2  Negativo     GESTIÓN DE SEGURIDAD Y SALUD EN EL TRABAJO   
3  Negativo     GESTIÓN DE SEGURIDAD Y SALUD EN EL TRABAJO   
4  Negativo     GESTIÓN DE SEGURIDAD Y SALUD EN EL TRABAJO   

                                         descripcion  \
0  Relación disponibilidad servicio infraestructu...   
1  Mide la eficien

In [5]:
# Filtrar campo 'Fecha' y eliminar registros vacíos
Reporte_Indicadores = Reporte_Indicadores.dropna(subset=['fecha'])

# Reemplazar '.' por ',' en la columna 'resultado'
if 'resultado' in Reporte_Indicadores.columns:
    Reporte_Indicadores['resultado'] = Reporte_Indicadores['resultado'].astype(str).str.replace('.', ',')

# Ajustar el campo 'clasificacion' reemplazando 'Estrat&eacute;gico' por 'Estratégico'
if 'clasificacion' in Reporte_Indicadores.columns:
    Reporte_Indicadores['clasificacion'] = Reporte_Indicadores['clasificacion'].replace('Estrat&eacute;gico', 'Estratégico')


<ipython-input-5-4c281c8b86ad>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Reporte_Indicadores['resultado'] = Reporte_Indicadores['resultado'].astype(str).str.replace('.', ',')
<ipython-input-5-4c281c8b86ad>:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Reporte_Indicadores['clasificacion'] = Reporte_Indicadores['clasificacion'].replace('Estrat&eacute;gico', 'Estratégico')


In [6]:
# Filtrar registros como 'Serie Unica'
Serie_Unica = Reporte_Indicadores[Reporte_Indicadores['Tipo'] == 'Serie Unica']

# Filtrar registros como 'Multiserie'
Multiserie = Reporte_Indicadores[Reporte_Indicadores['Tipo'] == 'Multiserie']

In [7]:
# Dividir la columna 'series' si existe
if 'series' in Multiserie.columns:
    def expand_json(row):
        try:
            series = ast.literal_eval(row)
            if isinstance(series, list):
                data = {}
                for i, item in enumerate(series):
                    data[f'meta_{i+1}'] = item.get('meta', '')
                    data[f'nombre_{i+1}'] = item.get('nombre', '')
                    data[f'resultado_{i+1}'] = item.get('resultado', '')
                    for j, var in enumerate(item.get('variables', [])):
                        data[f'variable_{i+1}_{j+1}'] = var.get('valor', '')
                        data[f'nombre_variable_{i+1}_{j+1}'] = var.get('nombre', '')
                        data[f'simbolo_variable_{i+1}_{j+1}'] = var.get('simbolo', '')
                return pd.Series(data)
        except (ValueError, SyntaxError):
            return pd.Series()

    expanded_columns = Multiserie['series'].apply(expand_json)
    Multiserie = pd.concat([Multiserie, expanded_columns], axis=1)


In [10]:
# Consolidar las dos bases
Consolidado = pd.concat([Serie_Unica, Multiserie], ignore_index=True)

# Crear la columna 'llave' concatenando 'ID' y 'Fecha' unidos por '-'
if 'ID' in Consolidado.columns and 'fecha' in Consolidado.columns:
    Consolidado['llave'] = Consolidado['ID'].astype(str) + '-' + Consolidado['fecha'].astype(str)
    Consolidado.insert(0, 'llave', Consolidado.pop('llave'))

<ipython-input-10-837d1cb38133>:6: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  Consolidado['llave'] = Consolidado['ID'].astype(str) + '-' + Consolidado['fecha'].astype(str)
<ipython-input-10-837d1cb38133>:7: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  Consolidado.insert(0, 'llave', Consolidado.pop('llave'))


In [11]:
# Guardar los nuevos DataFrames en archivos Excel separados
Serie_Unica.to_excel('Serie_Unica.xlsx', index=False)
Multiserie.to_excel('Multiserie.xlsx', index=False)
Consolidado.to_excel('Consolidado.xlsx', index=False)

print("Nuevo archivo guardado: Reporte_Indicadores_Filtrado.xlsx")
print("Archivo guardado: Serie_Unica.xlsx")
print("Archivo guardado: Multiserie.xlsx")
print("Archivo guardado: Consolidado.xlsx")

Nuevo archivo guardado: Reporte_Indicadores_Filtrado.xlsx
Archivo guardado: Serie_Unica.xlsx
Archivo guardado: Multiserie.xlsx
Archivo guardado: Consolidado.xlsx
